# Crawling Part
In this project we will predict if the costumer review on the resturant will be good or bad. We will learn about different parameters of comments by analize the word in it.
In this section we will deal with Crawling.

Our data aquision is devided to 2 parts - resturants and comments. Our data will be collected from <https://easy.co.il>.

## Importing Libraries
the next block will be responsible for installing and importing the necessary libraries and functions for this section.

download necesery packages

In [ ]:
#pip install selenium

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import math
import time
import random
import datetime

#### Setup Selenium initial definitions

In [ ]:
driver = webdriver.Chrome()
base_url = "https://easy.co.il/list/Restaurants-and-Fast-Food"
driver.get(base_url)

### Initial 2 DataFrames
**1) Resturants DF**- will contain data on each resturant

**2) Comments DF** - will contain the comments we found in each resturant comments review


In [ ]:
resturants_df = pd.DataFrame()
comments_df = pd.DataFrame()

In the next section we will get the data for the resturant df with Selenium

In [ ]:
def get_resturant_name():
    name = ""
    try:
        name = driver.find_element(by=By.CLASS_NAME, value= 'biz-title')
        if('\n' in name.text):
            name = name.text[0 : name.text.index('\n')]
        else:
            name = name.text
    except:
        name = None
    return name

def get_resturant_phone():
    phone = ""
    try:
        phone = driver.find_element(by=By.ID, value= 'action-phone-label').text
    except:
        phone=None
    return phone

def get_resturant_address():
    address = ""
    try:
        address = driver.find_element(by=By.CLASS_NAME, value= 'biz-address-text').text
    except:
        address=None
    return address

def get_opening_hours():
    hours = ""
    try:
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'open-hours')))
        button.click()
        hours = driver.find_element(by=By.CLASS_NAME, value="open-hours-content").text
        button.click()
    except:
        hours = None
    return hours

def get_resturant_type():
    resturant_type = ""
    try:
        resturant_type = driver.find_element(by=By.CLASS_NAME, value= 'best-sub-cat').text
    except:
        resturant_type = None
    return resturant_type

def get_resturant_rating():
    rating = 0
    try:
        rating = driver.find_elements(by=By.CLASS_NAME, value= 'easy-score')[1].text
    except:
        rating = -1
    return rating

def get_is_resturant_has_deliveries():
    try:
        return "משולחים" in driver.find_element(by=By.CLASS_NAME, value= 'upper-buttons-scroll-box').text
    except:
        return False
    
def get_is_resturant_has_table_reservation():
    try:
        return "הזמנת שולחן" in driver.find_element(by=By.CLASS_NAME, value= 'upper-buttons-scroll-box').text
    except:
        return False

def get_is_resturant_has_menu():
    try:
        return "תפריט" in driver.find_element(by=By.CLASS_NAME, value= 'upper-buttons-scroll-box').text
    except:
        return False
    
def get_is_resturant_has_sending_a_massage():
    try:
        return "הודעה" in driver.find_element(by=By.CLASS_NAME, value= 'upper-buttons-scroll-box').text
    except:
        return False
    
def get_element_item_from_list(list, text_to_search):
    for index, item in enumerate(list):
        if(text_to_search in item.text):
            return item

def get_url(what_to_get):
    link = ""
    try:
        lst = driver.find_elements(by=By.CLASS_NAME, value= 'biz-action-button')
        face=get_element_item_from_list(lst,what_to_get).get_attribute('innerHTML')
        position = -1
        for i in range(0, 2):
            position = face.find('"', position + 1)

        link = face[face.index('"') + 1 : position]
    except:
        link = None
    return link

def get_resturant_positive_reviews_count():
    positive_reviews_count = 0
    try:
        positive_reviews = driver.find_element(by=By.CLASS_NAME, value='green')
        positive_reviews_count = positive_reviews.text[ positive_reviews.text.index('(') + 1 : positive_reviews.text.index(')') ]
    except:
        positive_reviews_count = -1
    return positive_reviews_count

def get_resturant_negative_reviews_count():
    negative_reviews_count = 0
    try:
        negative_reviews = driver.find_element(by=By.CLASS_NAME, value='red')
        negative_reviews_count = negative_reviews.text[ negative_reviews.text.index('(') + 1 : negative_reviews.text.index(')') ]
    except:
        negative_reviews_count = -1
    return negative_reviews_count

def get_resturant_expert_reviews_count():
    expert_reviews_count = 0
    try:
        expert_reviews = driver.find_element(by=By.CLASS_NAME, value='yellow')
        expert_reviews_count = expert_reviews.text[ expert_reviews.text.index('(') + 1 : expert_reviews.text.index(')') ]
    except:
        expert_reviews_count = -1
    return expert_reviews_count

def get_resturant_side_data(what_to_search):
    result = 0
    try:
        parentElement = driver.find_element(by=By.CLASS_NAME, value="biz-card-items")
        elementsList = parentElement.find_elements(by=By.CLASS_NAME, value="card-text")
        for index,item in enumerate(elementsList):
            text=item.get_attribute('innerText')
            count=text[0 : text.index('\n')]
            what = text[text.index('\n')+1:].strip()
            if(what_to_search in what):
                result = count
    except:
        result = -1
    return result

def get_is_resturant_has_discounts_and_benefits():
    try:
        driver.find_element(by=By.CLASS_NAME, value="biz-benefits")
        return True
    except:
        return False
    
def get_resturant_postal_code():
    postal_code=0
    try:
        elementsList = driver.find_elements(by=By.CLASS_NAME, value="info-item")
        for index,item in enumerate(elementsList):
            if('מיקוד' in item.text):
                postal_code=item.text[item.text.index(' ') + 1 : ]
    except:
        postal_code = -1
    return postal_code

In the next section we will get the data for the comments df with Selenium

In [ ]:
def get_customer_name(item):
    customer_name= ""
    try:
        customer_name = item.find_element(by=By.CLASS_NAME, value= 'review-name').text
    except:
        customer_name= None
    return customer_name

def get_customer_reviews_number(item):
    customer_reviews_number = ""
    try:
        customer_reviews_number = item.find_element(by=By.CLASS_NAME, value= 'easy-reviews-count')
        customer_reviews_number= customer_reviews_number.text[0:customer_reviews_number.text.index(' ')]
    except:
        customer_reviews_number = None
    return customer_reviews_number

def get_customer_reviews_date(item):
    customer_reviews_date = ""
    try:
        customer_reviews_date = item.find_element(by=By.CLASS_NAME, value= 'review-date').text
    except:
        customer_reviews_date = None
    return customer_reviews_date

def get_comment(item):
    comment=""
    try:
        comment = item.find_element(by=By.CLASS_NAME, value= 'review-text').find_element(by=By.CSS_SELECTOR, value= 'p').text
    except:
        comment= None
    return comment

def get_number_of_star(item):
    number_of_star=""
    try:
        parentImg = item.find_element(by=By.CLASS_NAME, value="review-right")
        img = parentImg.find_element(by=By.CLASS_NAME, value='review-icon')
        src = img.get_attribute('src')
        number_of_star = src[46] 
    except:
        number_of_star= None
    return number_of_star

The comments of each resturant place in multiple pages.

In this Section we are calling the crawling function for *comments* and navigate to the next page after finishing the current page.

We saw that on each page there are 15 comments and we get the amount of comments for the resturant, we calculate how mush pages are avaliable for the resturant and navigate accordingly (for loop)

**Output**: all the comments of the resturant in DataFrame

In [ ]:
def get_all_comments(base_url):
    resturant_name=[]
    customer_name=[]
    customer_reviews_number=[]
    customer_reviews_date=[]
    comments=[]
    number_number_of_star=[]
    total_reviews = 0
    active_revies = driver.find_element(by=By.CLASS_NAME, value="reviews-tabs").find_element(by=By.CLASS_NAME, value="active")
    if("ביקורות" in active_revies.text):
        total_reviews = int(active_revies.text[active_revies.text.index('(')+1 : active_revies.text.index(')')])
    total_pages = math.ceil(total_reviews/15)
    for page_number in range(1, total_pages + 1):
        try:
            driver.get(base_url + "?reviewpage=" + str(page_number))
            parentElement = driver.find_element(by=By.CLASS_NAME, value="reviews-items")
            elementsList = parentElement.find_elements(by=By.CLASS_NAME, value="easy-review")
            for index,item in enumerate(elementsList):
                resturant_name.append(get_resturant_name())
                customer_name.append(get_customer_name(item))
                customer_reviews_number.append(get_customer_reviews_number(item))
                customer_reviews_date.append(get_customer_reviews_date(item))
                comments.append(get_comment(item))
                number_number_of_star.append(get_number_of_star(item))
            sleep_time = random.randint(1, 10)
            time.sleep(sleep_time)
        except Exception as e:
            print('no success',e)
    df=pd.DataFrame({"resturant_name":resturant_name,"customer_name":customer_name,"customer_reviews_number":customer_reviews_number,"customer_reviews_date":customer_reviews_date,"comments":comments,"number_number_of_star":number_number_of_star})
    return df

In this Section we are calling the crawling function for *resturant* **without comments**.

**Output**: resturant data object

In [ ]:
def get_resturant_data():
    return {'Name': get_resturant_name(),
            'Phone': get_resturant_phone(),
            'Address': get_resturant_address(),
            'Opening Hours': get_opening_hours(),
            'Type': get_resturant_type(),
            'Rating': get_resturant_rating(),
            'Has Deliveries': get_is_resturant_has_deliveries(),
            'Has Table Reservation': get_is_resturant_has_table_reservation(),
            'Has Menu': get_is_resturant_has_menu(),
            'Has Sending A Massage': get_is_resturant_has_sending_a_massage(),
            'Facebook Url': get_url("פייסבוק"),
            'Instagram Url': get_url("אינסטגרם"),
            'Home Page Url': get_url("אתר הבית"),
            'Positive Reviews Count': get_resturant_positive_reviews_count(),
            'Negative Reviews Count': get_resturant_negative_reviews_count(),
            'Expert Reviews Count': get_resturant_expert_reviews_count(),
            'Number Of Visits In Last Month': get_resturant_side_data("ביקרו פה החודש"),
            'Number Of Reviews': get_resturant_side_data("ביקורות"),
            'Number Of Top 3': get_resturant_side_data("TOP 3"),
            'Has Discounts And Benefits': get_is_resturant_has_discounts_and_benefits(),
            'Postal Code': get_resturant_postal_code()}
    

When we are opening the website and search for location, we want to see all the resturants around.

We need to scrool down the scroller to get the resturants.

The next section will scroll down the page until no more resturants found around the selected location

In [ ]:
def change_location(new_location):
    WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CLASS_NAME, 'change-location'))).click()
    driver.find_element(by=By.ID, value="searchAddress").send_keys(new_location)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'biz-0'))).click()

def scroll_page():
    but = driver.find_element(by=By.ID, value="nextPageButton")
    driver.execute_script("arguments[0].scrollIntoView(true);", but)    
    
def show_all_resturants():
    is_continue = True
    count = 1
    while is_continue:
        try:
            scroll_page()
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "nextPageButton")))
            button = WebDriverWait(driver, 10, 10).until(EC.element_to_be_clickable((By.ID, 'nextPageButton')))
            button.click()
            count = count + 1
        except:
            is_continue = False

We encountered difficulties during the crawling, were blocked dozens of times so we kept the restaurant links aside and ran the operation on a small portion of files each time

The next sections will present the link save for "תל אביב" area and "ראשון לציון" area

In [ ]:
def save_all_urls(file_name):
    all_resturants=[]
    current_resturant = ""
    count = 1
    try:
        elementsList = driver.find_elements(by=By.CLASS_NAME, value="biz-item")
        for index,item in enumerate(elementsList):
            resturant_url=item.find_element(by=By.CLASS_NAME, value="biz-item-link-wrapper").get_attribute('href')
            if("=" in resturant_url):
                resturnat_number = resturant_url[resturant_url.index("=") + 1 : resturant_url.index("&")]
                all_resturants.append("https://easy.co.il/page/" + resturnat_number)
            else:
                all_resturants.append(resturant_url)

        df=pd.DataFrame({"resturant_url":all_resturants})
        df.to_csv(file_name)

       
    except Exception as e:
        print('error', current_resturant)
        print(e)
        print('---------')

In [ ]:
change_location("תל אביב")
# show_all_resturants()
# save_all_urls("tel_aviv_urls.csv")
# change_location("ראשון לציון")
# show_all_resturants("rishon_letzion_urls.csv")
# save_all_urls()


The next section is the **main** section of our crawling.

As we say earlier the resturants_url is taken from file we saved before in `save_all_urls()` func

Foreach resturant in the file we create df with the data and add it to the current df.

After we finish the crawling on each block of resturants we save it to the local storage

In [ ]:
all_resturants=pd.read_csv("rishon_letzion_urls.csv")
# all_resturants=pd.read_csv("tel_aviv_urls.csv")
current_resturant = ""
count = 1
try:
    for rest in all_resturants["resturant_url"][count:count+20]:
        current_resturant = rest
        driver.get(rest)
        resturants_df = resturants_df.append(get_resturant_data(), ignore_index=True)
        comments_df = pd.concat([comments_df, get_all_comments(rest)])
        sleep_time = random.randint(10, 30)
        time.sleep(sleep_time)
        count = count + 1
except Exception as e:
    print('error', current_resturant)
    print(e)
    print('---------')

driver.close()
resturants_df.to_csv('resturants1.csv', encoding = 'utf-8-sig')
comments_df.to_csv('comments1.csv', encoding = 'utf-8-sig')